# Generating random charts with `Altair` and `Scipy`

Based on _The Weighted Average Illusion:  Biases in Perceived Mean Position in Scatterplots_, the authors used the following: 

> To generate the x- and y-data, we used Poisson disk sampling [50] to produce 30 uniquely distributed point grids, with minimum distance between the boundaries of any two points set at 8 pixels. This methodology is similar to Gleicher et al. [34]. Each dataset always contained 30 marks, with the number of points selected in piloting.

For the sake of integrating with the `Revisit` platformm, we will use `Altair` and `Scipy`. Luckily `scipy` provides a possion-disk sampler. 

- Note that QMC only provide an $n \times d$ array of numbers in $[0, 1]$. ([source](https://docs.scipy.org/doc/scipy/reference/stats.qmc.html))

In [1]:
# import altair as alt 
import polars as pl
from scipy.stats import qmc # quasi monte carlo submodule 
import numpy as np
import altair as alt

## Playing with the basic syntax 

In [2]:
# Poisson disk sampling 
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.PoissonDisk.html

rng = np.random.default_rng()
engine = qmc.PoissonDisk(d=2, radius = 0.08, rng = None)
sample = engine.random(30).round(2) # round this to 2 

# turn sample into df for plotting
df = pl.DataFrame(sample)
df
# df.head(5)

# plot using altair
(
    
    alt.Chart(df).mark_point(filled = True).encode(
        alt.X('column_0').scale(domain=(0, 1)),
        alt.Y('column_1').scale(domain=(0, 1))
    ).properties(
        # width = 300,
        # height = 300
    )
)

alt.Chart(...)

The default widht and height of altair is **300 pixels**. The default `mark_point` size (**pixel area**, NOT radius) is 30 pixels. 

## Define plotting function 

In [11]:
def create_plot(n=30, point_radius=12.5, point_gap=8, canvas_size=500, optimization_on=True):
    # initialize poisson engine 
    engine = qmc.PoissonDisk(d=2, 
                             radius = (point_radius * 2 + point_gap) * 1.05 / canvas_size, 
                             hypersphere="volume", 
                             ncandidates = 30, 
                             optimization="lloyd" if optimization_on else None)
    # sample dots and scale them 
    sample = qmc.scale(engine.random(n), [point_radius, point_radius], [canvas_size - point_radius, canvas_size - point_radius])
    # turn it to dataframe 
    df = pl.DataFrame(sample)
    # turn it to chart 
    chart = alt.Chart(df).mark_point(filled = True, size = point_radius * point_radius * np.pi, color = "gray").encode(
        alt.X('column_0').scale(domain=(0, canvas_size)).axis(labels = False, grid = False, title = None),
        alt.Y('column_1').scale(domain=(0, canvas_size)).axis(labels = False, grid = False, title = None)
    ).properties(
        width = canvas_size,
        height = canvas_size
    )
    return chart, df, df.mean() # df.mean().to_numpy().flatten()

In [12]:
# optimization makes the points spread more evenly across the canvas 
create_plot()[0]

alt.Chart(...)

In [13]:
create_plot()[1]
create_plot()[2]

column_0,column_1
f64,f64
196.123503,357.47793


In [38]:
# or else the dots look more condensed 
create_plot(optimization_on=True)[0]

alt.Chart(...)

## Outputing things into vega spec

Using `vl-convert`: https://github.com/vega/vl-convert

In [41]:
import vl_convert as vlc

In [58]:
c = create_plot(optimization_on=False)[0]
c.to_json()
# c.to_url()

'{\n  "$schema": "https://vega.github.io/schema/vega-lite/v6.1.0.json",\n  "config": {\n    "view": {\n      "continuousHeight": 300,\n      "continuousWidth": 300\n    }\n  },\n  "data": {\n    "name": "data-9f9a9a98be21da23073d6365a6753d13"\n  },\n  "datasets": {\n    "data-9f9a9a98be21da23073d6365a6753d13": [\n      {\n        "column_0": 121.3751121191075,\n        "column_1": 90.42235308107236\n      },\n      {\n        "column_0": 84.48356099789294,\n        "column_1": 50.38890908857327\n      },\n      {\n        "column_0": 149.85152724778547,\n        "column_1": 123.40174769836771\n      },\n      {\n        "column_0": 170.06423303891458,\n        "column_1": 54.025482169939046\n      },\n      {\n        "column_0": 128.03557825721407,\n        "column_1": 28.031751800779823\n      },\n      {\n        "column_0": 107.72231713799856,\n        "column_1": 148.9286490394005\n      },\n      {\n        "column_0": 59.76621764190779,\n        "column_1": 74.7052141435505\n   

In [59]:
vega_spec = vlc.vegalite_to_vega(c.to_json(), vl_version="5.20")
vega_spec

{'$schema': 'https://vega.github.io/schema/vega/v5.json',
 'background': 'white',
 'padding': 5,
 'width': 500,
 'height': 500,
 'style': 'cell',
 'data': [{'name': 'data-9f9a9a98be21da23073d6365a6753d13',
   'values': [{'column_0': 121.3751121191075, 'column_1': 90.42235308107236},
    {'column_0': 84.48356099789294, 'column_1': 50.38890908857327},
    {'column_0': 149.85152724778547, 'column_1': 123.40174769836771},
    {'column_0': 170.06423303891458, 'column_1': 54.025482169939046},
    {'column_0': 128.03557825721407, 'column_1': 28.031751800779823},
    {'column_0': 107.72231713799856, 'column_1': 148.9286490394005},
    {'column_0': 59.76621764190779, 'column_1': 74.7052141435505},
    {'column_0': 88.74246908967139, 'column_1': 96.71278233514136},
    {'column_0': 184.31765450571743, 'column_1': 105.58158448768936},
    {'column_0': 208.49277775041776, 'column_1': 141.81176445679608},
    {'column_0': 174.7535316991381, 'column_1': 160.3117382706327},
    {'column_0': 124.86986

In [61]:
import json 
vega_json_string = json.dumps(vega_spec, indent=2)
with open('spec.json', 'w') as f:
    json.dump(vega_spec, f, indent=2)

## Calculate mean 

In [209]:
a, b, c = create_plot(optimization_on=False)

In [210]:
a

alt.Chart(...)

In [212]:
# Add centroid afterward
centroid_mark = alt.Chart(c).mark_circle(
    size=15**2 * np.pi,
    color='red',
    stroke='black',
    strokeWidth=2
).encode(
    x=alt.X('column_0', scale=alt.Scale(domain=[0, 500]), axis=None),
    y=alt.Y('column_1', scale=alt.Scale(domain=[0, 500]), axis=None)
)

# Layer them together
chart_with_centroid = a + centroid_mark

In [213]:
chart_with_centroid

alt.LayerChart(...)